# YASA

This notebook demonstrates how to apply the detection only on **NREM sleep** by providing an hypnogram (= sleep stage vector) to YASA.

Please make sure to install YASA first by typing the following line in your terminal or command prompt:

`pip install --upgrade yasa`


## Load the data and hypnogram

We first load a full-night 3-channels dataset (Cz, Fz, Pz) sampled at 100 Hz. The data is in compressed NumPy format (*.npz*).

In [1]:
import yasa
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(context='notebook', font_scale=1.3)

# Load data
f = np.load('data_full_6hrs_100Hz_Cz+Fz+Pz.npz')
data, ch_names = f['data'], f['chan']
sf = 100.
times = np.arange(data.size) / sf

print(data.shape, ch_names)
print(np.round(data[:, 0:5], 3))

(3, 2161058) ['Cz' 'Fz' 'Pz']
[[15.797 22.307 39.922 25.657 27.094]
 [16.896 26.385 40.966 21.833 24.456]
 [ 5.899 14.297 36.592 26.094 23.395]]


**Hypnogram**

Next, we load the sleep staging vector (a.k.a hypnogram), which is a simple text file in which each value represents 30 seconds of data. Sleep stages are encoded as integer (*0: Wake, 1: N1 sleep, 2: N2 sleep, 3: N3 sleep, 4: REM sleep*).

In the code below, we load our 30-sec hypnogram and upsample it to match the sampling frequency and length of data, using YASA's built-in [hypno_upsample_to_data](https://htmlpreview.github.io/?https://raw.githubusercontent.com/raphaelvallat/yasa/master/html/hypno.html#yasa.hypno.hypno_upsample_to_data) function.

Please refer to [10_bandpower.ipynb](10_bandpower.ipynb) for more details on how to manipulate hypnogram in YASA.

In [2]:
hypno_30s = np.loadtxt('data_full_6hrs_100Hz_hypno_30s.txt')
hypno = yasa.hypno_upsample_to_data(hypno=hypno_30s, sf_hypno=(1/30), data=data, sf_data=sf)
print(hypno.shape, 'Unique values =', np.unique(hypno))

(2161058,) Unique values = [0. 1. 2. 3. 4.]


c:\users\raphael\desktop\yasa\yasa\hypno.py:160: UserWarning: Hypnogram is SHORTER then data by 10.58 seconds. Padding hypnogram (with last value) to match data.size.
  'data.size.' % dur_diff)


*************

**Applying the detection on NREM sleep only**

To apply the multi-channel detection, we use the `yasa.spindles_detect_multi` function. Please check [01_spindles_detection_multi.ipynb](01_spindles_detection_multi.ipynb) for more details. We also pass the hypnogram vector to restrain the detection to NREM sleep epochs (stage 1, 2 and 3).

In [3]:
sp = yasa.spindles_detect_multi(data, sf, ch_names=ch_names, hypno=hypno)
print(sp.shape[0], 'spindles detected.')
sp.head().round(3)

1669 spindles detected.


,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Stage,Channel,IdxChannel
0,522.55,523.79,1.24,45.569,10.891,2.133,0.470,13.079,16.0,0.184,1,Cz,0
1,585.54,586.38,0.84,59.683,13.335,2.257,0.476,12.958,10.0,0.647,2,Cz,0
2,598.08,599.52,1.44,79.824,16.180,2.376,0.423,12.894,19.0,0.372,2,Cz,0
3,604.37,605.09,0.72,60.745,12.820,2.220,0.355,12.648,9.0,0.493,2,Cz,0
4,607.53,608.05,0.52,50.211,13.847,2.358,0.258,13.452,6.0,0.849,2,Cz,0


In [4]:
# We now print the number of spindles detected per channel and sleep stage
sp.groupby(['Channel', 'Stage'])['Start'].count()

Channel  Stage
Cz       1         22
         2        536
         3         30
Fz       1         17
         2        432
         3         19
Pz       1         24
         2        569
         3         20
Name: Start, dtype: int64

**Detection on N2 and N3 sleep only (exclude N1)**

In [5]:
sp = yasa.spindles_detect_multi(data, sf, ch_names=ch_names, hypno=hypno, include=(2, 3))
print(sp.shape[0], 'spindles detected.')
sp.head().round(3)

1604 spindles detected.


,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Stage,Channel,IdxChannel
0,585.54,586.38,0.84,59.683,13.335,2.257,0.476,12.958,10.0,0.647,2,Cz,0
1,598.09,599.52,1.43,79.858,16.234,2.377,0.424,12.889,19.0,0.368,2,Cz,0
2,604.37,605.08,0.71,60.637,12.875,2.222,0.355,12.648,9.0,0.500,2,Cz,0
3,607.53,608.05,0.52,50.211,13.847,2.358,0.258,13.452,6.0,0.849,2,Cz,0
4,622.93,624.39,1.46,144.872,26.808,2.651,0.466,12.769,18.0,0.408,2,Cz,0


**Detection on the whole recording**

In [6]:
sp = yasa.spindles_detect_multi(data, sf, ch_names=ch_names)
print(sp.shape[0], 'spindles detected.')
sp.head().round(3)

1919 spindles detected.


,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Channel,IdxChannel
0,522.54,523.79,1.25,45.509,10.850,2.131,0.470,13.083,16.0,0.190,Cz,0
1,585.51,586.38,0.87,59.435,13.185,2.255,0.473,12.946,11.0,0.659,Cz,0
2,598.06,599.54,1.48,79.922,15.977,2.373,0.421,12.889,19.0,0.376,Cz,0
3,604.36,605.12,0.76,60.649,12.520,2.209,0.350,12.646,9.0,0.481,Cz,0
4,607.53,608.05,0.52,50.211,13.847,2.358,0.258,13.452,6.0,0.849,Cz,0
